In [ ]:
import json
from pathlib import Path

In [ ]:
keylabs_labels_eo = Path("/Users/danielschmid/projects/PromptAdherence/markdown_generation/500_selected_images/anduril_eo_500_selected_demo_images/keylabs_labels")
keylabs_labels_ir = Path("/Users/danielschmid/projects/PromptAdherence/markdown_generation/500_selected_images/anduril_ir_500_selected_demo_images/keylabs_labels")
known_labels = [
    "ship",
    "boat",
    "buoy",
    "bird",
    "unknown",
    "jetski",
    "kayak"
]



In [ ]:
def to_label(n_ships: int, n_buoys: int, n_birds: int, n_unknown: int):
    if n_ships > 0:
        if n_ships <= 4:
            return "1-4_ships"
        
        if n_ships <= 9:
            return "5-9_ships"

        if n_ships <= 14:
            return "10-14_ships"
        
        return "15+_ships"

    else:
        if n_buoys > 0 or n_birds > 0:
            return "bird_or_buoy"
        else:
            return "no_objects"

def process_keylabs_labels(labels_dir: Path, img_name_to_label_map: dict) -> dict:
    for label_file in labels_dir.glob("*.json"):
        with open(label_file, "r") as f:
            label_data = json.load(f) 
        
        label_metadata = label_data[0]
        label_objects = label_data[0]['objects']

        for label in label_objects: 
            if label['type'] not in known_labels:
                raise ValueError(f"Unknown label type: {label['type']}")
        
        n_ships = len([x for x in label_objects if x['type'] == 'ship' or x['type'] == 'boat' or x['type'] == 'jetski' or x['type'] == 'kayak'])
        n_buoys = len([x for x in label_objects if x['type'] == 'buoy'])
        n_birds = len([x for x in label_objects if x['type'] == 'bird'])
        n_unknown = len([x for x in label_objects if x['type'] == 'unknown'])
        
        img_name = Path(label_metadata['file']).name
        img_label = to_label(n_ships, n_buoys, n_birds, n_unknown)
        img_name_to_label_map[str(img_name)] = img_label
    return img_name_to_label_map


img_name_to_label_map_eo = {
    str(Path(json.load(Path(x).open())[0]['file']).name): {} for x in keylabs_labels_eo.glob("*.json")
}
result = process_keylabs_labels(keylabs_labels_eo, img_name_to_label_map_eo)

In [ ]:
result

In [ ]:
eo_labels_path = Path("merged_labels_eo.jsonl")
ir_labels_path = Path("merged_labels_ir.jsonl")


eo_labels = [json.loads(line) for line in eo_labels_path.open()]
ir_labels = [json.loads(line) for line in ir_labels_path.open()]

def log_statistics(labels: list[dict]):
    counter = {}

    for label in labels:
        label_str = f"{label['uav_or_usv']}_{label['open_water_or_coastline']}_{label['ship_type']}"
        if label_str not in counter.keys():
            counter[label_str] = 0
        counter[label_str] += 1
   
    for label_str, count in counter.items():
        print(f"{label_str}: {count}")

print("EO")
log_statistics(eo_labels)
print("="*100)
print("IR")
log_statistics(ir_labels)


EO
usv_open_water_1-4_ships: 62
usv_open_water_no_objects: 9
usv_coastline_no_objects: 8
usv_coastline_1-4_ships: 25
usv_coastline_5-9_ships: 1
uav_coastline_1-4_ships: 178
uav_open_water_1-4_ships: 187
uav_coastline_no_objects: 11
uav_open_water_no_objects: 6
uav_coastline_5-9_ships: 5
uav_coastline_bird_or_buoy: 5
uav_open_water_5-9_ships: 2
uav_coastline_10-14_ships: 1
IR
usv_open_water_1-4_ships: 10
usv_coastline_1-4_ships: 10
usv_coastline_no_objects: 22
usv_coastline_bird_or_buoy: 4
usv_open_water_no_objects: 5
usv_open_water_bird_or_buoy: 1
uav_open_water_1-4_ships: 175
uav_coastline_no_objects: 74
uav_open_water_5-9_ships: 10
uav_coastline_1-4_ships: 154
uav_open_water_no_objects: 21
uav_coastline_bird_or_buoy: 3
uav_coastline_5-9_ships: 10
uav_coastline_10-14_ships: 1
